### Importing and setting up the client to interact with carla server

In [ ]:
import carla
import time
import math
import random
import numpy as np
import cv2

In [2]:
# Call back function to capture and save image
def process_image(image):
    # Convert the image to a NumPy array
    image_array = np.array(image.raw_data)
    image_array = image_array.reshape((image.height, image.width, 4))
    image_array = image_array[:, :, :3]
    cv2.imwrite('/home/mudit/independent_study/cam_cap/camera0_image' + str(time.time()) + '.png', image_array)

def segment_callback(image):
    image.convert(carla.ColorConverter.CityScapesPalette)
    img_2 = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    img_2 = img_2[:, :, :3]
    cv2.imwrite('sem_seg/semantic_seg_img_' + str(time.time()) + '.png', img_2)

# Callback stores sensor data in a dictionary for use outside callback                         
def seg_camera_callback(image, data_dict):
#     image.convert(carla.ColorConverter.CityScapesPalette)
    img_2 = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    img_2 = img_2[:, :, :3]
    img_2 = 
    print(img_2.min())
    print(img_2.max())
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))



SyntaxError: invalid syntax (4089226510.py, line 20)

In [14]:
client = carla.Client('localhost', 2000)
world = client.get_world()

In [15]:
bp_library = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

### Spawnning a vehicle and setting spectator camera at appropriate position

In [16]:
spec_init_transform = carla.Transform(carla.Location(x=1.0, y=0.0, z=200.4), carla.Rotation(pitch=-90.0, yaw=0.0, roll=0.0))
spectator = world.get_spectator()
spectator.set_transform(spec_init_transform)

In [11]:
vehicle_bp = bp_library.find('vehicle.lincoln.mkz_2020')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

In [12]:
spectator = world.get_spectator()
veh_transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2)),vehicle.get_transform().rotation)
spectator.set_transform(veh_transform)

### Spawn NPC vehicles and set autopilot to true to make them move autonomously

In [13]:
# Spawn dummy vehicles in the world that act as per the carla sim autopilot
for i in range(15):
    vehicle_bp = random.choice(bp_library.filter('vehicle'))
    npc_veh = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
    npc_veh.set_autopilot(True)

In [14]:
# Set behaviour of the vehicles in the simulation to autopilot of carla sim
for veh in world.get_actors().filter('*vehicle*'):
	veh.set_autopilot(True)


### RGB camera

In [9]:
camera_bp = bp_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '110')

# Set the time in seconds between sensor captures
camera_bp.set_attribute('sensor_tick', '1.0')

camera_init_transform = carla.Transform(carla.Location(x=1.0, y=0.0, z=1.4), carla.Rotation(pitch=0.0, yaw=0.0, roll=0.0))
camera = world.spawn_actor(camera_bp, camera_init_transform, attach_to=vehicle)

In [10]:
camera.id

29

In [12]:
camera.listen(process_image)

In [13]:
camera.stop()
camera.destroy()

### Semantic segmentation camera

In [35]:
seg_camera_bp = bp_library.find('sensor.camera.semantic_segmentation')
seg_camera_bp.set_attribute('image_size_x', '1280')
seg_camera_bp.set_attribute('image_size_y', '720')
seg_camera_bp.set_attribute('fov', '110')

# Set the time in seconds between sensor captures
seg_camera_bp.set_attribute('sensor_tick', '1.0')

seg_camera_init_transform = carla.Transform(carla.Location(x=1.0, y=0.0, z=30), carla.Rotation(pitch=-90.0, yaw=0.0, roll=0.0))
seg_camera = world.spawn_actor(seg_camera_bp, seg_camera_init_transform, attach_to=vehicle)

In [20]:
seg_camera.listen(segment_callback)

In [36]:
# Get ccamera dimensions and initialise dictionary                       
image_w = seg_camera_bp.get_attribute("image_size_x").as_int()
image_h = seg_camera_bp.get_attribute("image_size_y").as_int()
seg_camera_data = {'image': np.zeros((image_h, image_w, 4))}

# Start camera recording
seg_camera.listen(lambda image: seg_camera_callback(image, seg_camera_data))

0
240


In [37]:
# OpenCV named window for rendering
cv2.namedWindow('segm Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('segm Camera', seg_camera_data['image'])
cv2.waitKey(1)

# Game loop
while True:
    
    # Imshow renders sensor data to display
    cv2.imshow('segm Camera', seg_camera_data['image'])
    
    # Quit if user presses 'q'
    if cv2.waitKey(1) == ord('q'):
        break

# Close OpenCV window when finished
cv2.destroyAllWindows()

0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240
0
240


In [38]:
seg_camera.stop()
seg_camera.destroy()

True

### Lidar data acquisition

In [16]:
import open3d as o3d
from matplotlib import cm

In [17]:
# Auxilliary code for colormaps and axes
VIRIDIS = np.array(cm.get_cmap('plasma').colors)
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])

COOL_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
COOL = np.array(cm.get_cmap('winter')(COOL_RANGE))
COOL = COOL[:,:3]

def add_open3d_axis(vis):
    """Add a small 3D axis on Open3D Visualizer"""
    axis = o3d.geometry.LineSet()
    axis.points = o3d.utility.Vector3dVector(np.array([
        [0.0, 0.0, 0.0],
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]]))
    axis.lines = o3d.utility.Vector2iVector(np.array([
        [0, 1],
        [0, 2],
        [0, 3]]))
    axis.colors = o3d.utility.Vector3dVector(np.array([
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]]))
    vis.add_geometry(axis)

In [18]:
# LIDAR callback
def lidar_callback(point_cloud, point_list):
    """Prepares a point cloud with intensity
    colors ready to be consumed by Open3D"""
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    data = np.reshape(data, (int(data.shape[0] / 4), 4))

    # Isolate the intensity and compute a color for it
    intensity = data[:, -1]
    intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))
    int_color = np.c_[
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 0]),
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 1]),
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 2])]

    points = data[:, :-1]

    points[:, :1] = -points[:, :1]

    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)

In [19]:
# Set up LIDAR parameters are to assisst visualisation
lidar_bp = bp_library.find('sensor.lidar.ray_cast') 
lidar_bp.set_attribute('range', '20.0')
lidar_bp.set_attribute('noise_stddev', '0.1')
lidar_bp.set_attribute('upper_fov', '15.0')
lidar_bp.set_attribute('lower_fov', '-15.0')
lidar_bp.set_attribute('channels', '64.0')
lidar_bp.set_attribute('rotation_frequency', '20.0')
lidar_bp.set_attribute('points_per_second', '500000')
    
lidar_init_trans = carla.Transform(carla.Location(z=2))
lidar = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=vehicle)


In [20]:
# Add auxilliary data structures
point_list = o3d.geometry.PointCloud()

# Start sensors
lidar.listen(lambda data: lidar_callback(data, point_list))

In [ ]:
# Open3D visualiser for LIDAR
vis = o3d.visualization.Visualizer()
vis.create_window(
    window_name='Carla Lidar',
    width=960,
    height=540,
    left=480,
    top=270)

vis.get_render_option().background_color = [0.05, 0.05, 0.05]
vis.get_render_option().point_size = 1
vis.get_render_option().show_coordinate_frame = True
add_open3d_axis(vis)

# Update geometry and camera in game loop
frame = 0
while frame < 10000S:
    if frame == 2:
        vis.add_geometry(point_list)
    vis.update_geometry(point_list)
    
    vis.poll_events()
    vis.update_renderer()
    # # This can fix Open3D jittering issues:
    time.sleep(0.005)
    frame += 1


# Close displayws and stop sensors
lidar.stop()
lidar.destroy()
vis.destroy_window()

### Remove spawned vehicles

In [4]:
for veh in world.get_actors().filter('*vehicle*'):
	veh.destroy()

In [14]:
# Disconnect the client from the carla sim server to cleanly exit simulation

client.disconnect()

AttributeError: 'Client' object has no attribute 'disconnect'

AttributeError: 'World' object has no attribute 'kill'